# Lesson 3: Exercises


## Exercise 1: Equilibrium reaction with intermediate state

We will consider again consider a batch reaction in a `CSTR` but this time, we will also account for an intermediate state:

$\require{mhchem}$
$$\ce{A <=>[k_{AB}][k_{BA}] B <=>[k_{BC}][k_{CB}] C}$$

***Task:*** Implement the reaction and plot the results. Assume the following values for the rate constants:
- $k_{AB} = 0.080~s^{-1}$
- $k_{BA} = 0.0~s^{-1}$
- $k_{BC} = 0.060~s^{-1}$
- $k_{CB} = 0.0~s^{-1}$

## Exercise 2: Equilibrium reaction with intermediate state in tubular reactor
***Task:*** Implement the reaction in a tubular reactor and plot the results at the outlet, as well as over the length of the column for the last timestep.

Consider again the reaction parameters from Exercise 1.
For the `TubularReactor` use the following parameters:
- length: 1 m
- diameter: 10 cm
- flow rate: 0.1 m³/s




***Hint:*** To plot the bulk solution, make sure that you set the `write_solution_bulk` flag in the `TubularReactor`.

## Bonus Exercise

Try implementing other reaction systems such as:

$\require{mhchem}$
$$\ce{A + B ->[k_{1}] C}$$
$$\ce{2 A + B ->[k_{1}] C}$$
$$\ce{A + B <=>[k_{1}][k_{-1}] C ->[k_{2}] D}$$
$$\ce{A + B ->[k_{1}] C} \quad \text{and as a parallel reaction} \quad \ce{A + C ->[k_{2}] D}$$